# VGG16 Multi-Task Model for MedMNIST Submission Using Kaggle Input Data (64×64 Resolution)

This notebook loads MedMNIST datasets directly from the Kaggle input folder, converts the images to 3-channel format and resizes them to 64×64 (to reduce memory usage), builds a VGG16-based model for each MedMNIST task (excluding ChestMNIST), trains each model for up to **100 epochs** with early stopping (monitoring validation loss), and generates a submission CSV file.  
The submission file is formatted with columns: `id`, `id_image_in_task`, `task_name`, and `label`.

*Note: The correct class name for the dermatoscope images is `DermaMNIST` (with a capital M).*

In [1]:
import tensorflow as tf

# Check for available GPUs and enable dynamic memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs found and configured:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Please switch your runtime to GPU in the Kaggle Notebook settings.")

GPUs found and configured: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
!pip install -q medmnist 

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import f1_score
import medmnist
from medmnist import INFO
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# We will pad the 28x28 images to 32x32 for VGG models.
TARGET_SIZE = (32, 32)  
BATCH_SIZE = 32

In [4]:
def load_npz_data(npz_path):
    """
    Load train, val, and test arrays from an NPZ file.
    Expected keys: 'train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels'
    """
    data = np.load(npz_path)
    return (data['train_images'], data['train_labels']), (data['val_images'], data['val_labels']), (data['test_images'], data['test_labels'])

def create_tf_dataset_from_numpy(images, labels, batch_size=BATCH_SIZE, augment=False):
    """
    Convert numpy arrays to a tf.data.Dataset:
      - Expand 28x28 grayscale images to shape (28,28,1) then tile to (28,28,3).
      - Pad the 28x28 images to 32x32.
      - Normalize pixel values.
      - Optionally apply augmentation.
    """
    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)  # (N,28,28,1)
    if images.shape[-1] == 1:
        images = np.tile(images, (1,1,1,3))          # (N,28,28,3)
    
    images = images.astype(np.float32) / 255.0

    def _process(image, label):
        # Pad from 28x28 to 32x32 (2 pixels on top, bottom, left, and right)
        image = tf.pad(image, paddings=[[0,0],[2,2],[2,2],[0,0]])
        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
        return image, label

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.map(_process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(images))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [5]:
# Define tasks (excluding ChestMNIST)
task_names = [
    "pathmnist",
    "dermamnist",   # Correct python_class in INFO is "DermaMNIST"
    "octmnist",
    "pneumoniamnist",
    "retinamnist",
    "breastmnist",
    "bloodmnist",
    "tissuemnist",
    "organamnist",
    "organcmnist",
    "organsmnist"
]

base_path = Path("/kaggle/input/tensor-reloaded-multi-task-med-mnist/data")
task_to_npz = {task: base_path / f"{task}.npz" for task in task_names}

train_datasets = {}
val_datasets = {}
test_datasets = {}

for task in task_names:
    (train_imgs, train_lbls), (val_imgs, val_lbls), (test_imgs, test_lbls) = load_npz_data(task_to_npz[task])
    train_datasets[task] = (train_imgs, train_lbls)
    val_datasets[task] = (val_imgs, val_lbls)
    test_datasets[task] = (test_imgs, test_lbls)
    print(f"{task}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test samples")

pathmnist: 89996 train, 10004 val, 7180 test samples
dermamnist: 7007 train, 1003 val, 2005 test samples
octmnist: 97477 train, 10832 val, 1000 test samples
pneumoniamnist: 4708 train, 524 val, 624 test samples
retinamnist: 1080 train, 120 val, 400 test samples
breastmnist: 546 train, 78 val, 156 test samples
bloodmnist: 11959 train, 1712 val, 3421 test samples
tissuemnist: 165466 train, 23640 val, 47280 test samples
organamnist: 34581 train, 6491 val, 17778 test samples
organcmnist: 13000 train, 2392 val, 8268 test samples
organsmnist: 13940 train, 2452 val, 8829 test samples


In [6]:
train_datasets_tf = {}
val_datasets_tf = {}
test_datasets_tf = {}

for task in task_names:
    train_imgs, train_lbls = train_datasets[task]
    val_imgs, val_lbls = val_datasets[task]
    test_imgs, test_lbls = test_datasets[task]
    
    train_datasets_tf[task] = create_tf_dataset_from_numpy(train_imgs, train_lbls, batch_size=BATCH_SIZE, augment=True)
    val_datasets_tf[task]   = create_tf_dataset_from_numpy(val_imgs, val_lbls, batch_size=BATCH_SIZE, augment=False)
    test_datasets_tf[task]  = create_tf_dataset_from_numpy(test_imgs, test_lbls, batch_size=BATCH_SIZE, augment=False)
    
    count = sum(1 for _ in test_datasets_tf[task])
    print(f"{task}: Test batches: {count}")

ValueError: in user code:

    File "<ipython-input-4-adc5d518a550>", line 26, in _process  *
        image = tf.pad(image, paddings=[[0,0],[2,2],[2,2],[0,0]])

    ValueError: Shape must be rank 4 but is rank 3 for '{{node Pad}} = Pad[T=DT_FLOAT, Tpaddings=DT_INT32](args_0, Pad/paddings)' with input shapes: [28,28,3], [4,2].


In [ ]:
def build_vgg16_model_28x28(num_classes, input_shape=(28,28,3)):
    inputs = keras.Input(shape=input_shape)
    # Pad input from 28x28 to 32x32
    padded = layers.Lambda(lambda x: tf.pad(x, [[0,0],[2,2],[2,2],[0,0]]))(inputs)
    
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))
    base_model.trainable = False
    x = base_model(padded, training=False)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage:
# num_classes = len(INFO["pathmnist"]['label'])
# model_vgg16 = build_vgg16_model_28x28(num_classes)
# model_vgg16.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

models = {}
histories = {}

for task in task_names:
    num_classes = len(INFO[task]['label'])
    print(f"\nTraining VGG16 model for {task} with {num_classes} classes...")
    model_task = build_vgg16_model_28x28(num_classes, input_shape=(28,28,3))
    history = model_task.fit(
        train_datasets_tf[task],
        validation_data=val_datasets_tf[task],
        epochs=100,
        callbacks=[early_stopping],
        verbose=1
    )
    models[task] = model_task
    histories[task] = history

In [ ]:
submission_rows = []
global_id = 0

for task in task_names:
    model_task = models[task]
    preds_list = []
    for images, _ in test_datasets_tf[task]:
        preds = model_task.predict(images)
        preds = np.argmax(preds, axis=1)
        preds_list.append(preds)
    preds_all = np.concatenate(preds_list)
    for idx, pred in enumerate(preds_all):
        submission_rows.append([global_id, idx, task, int(pred)])
        global_id += 1

submission_df = pd.DataFrame(submission_rows, columns=["id", "id_image_in_task", "task_name", "label"])
print("Total submission rows:", len(submission_df))
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")